### PadrePads custom loggers
To develop custom loggers, we need to write a class that inherits from the base class LoggingFunction. Then, those custom loggers can be mapped to events of the user choice in the parameter mapping of the PyPads class. 


```python
from pypads.functions.loggers.base_logger import LoggingFunction
class CustomLogger(LoggingFunction):
    """
    This class would represents the custom logger.
    """
    def __pre__(self, ctx, *args, _pypads_env, _args, _kwargs, **kwargs):
        # The custom code you want to execute before calling the tracked function.
        # ctx: can be either the class of the tracked function, the module, or None.
        # _args, _kwarg
        
    def __post__(self, ctx, *args, _pypads_env, _pypads_pre_return, _pypads_result, _args, _kwargs, **kwargs):
        # The custom code to be executed after the call of the tracked function where "_pypads_pre_return"
        # and "_pypads_result" are the return value of self.__pre__ and the tracked function respectively"
        
```

##### Example

Let's say we want to log the accuracy score from the previous example but we want to log other metrics that we can compute using the Truth and Predicted values along with it. This should be done in the __post__ method.


In [1]:
from pypads.functions.loggers.base_logger import LoggingFunction
import mlflow
from mlflow.utils.autologging_utils import try_mlflow_log
from pypads import logger

class custom_metric(LoggingFunction):
    def __post__(self, ctx, *args, _pypads_env, _pypads_pre_return, _pypads_result, _args, _kwargs, **kwargs):
        accuracy_score = _pypads_result
        # Logging the accuracy score as an mlflow metric.
        if isinstance(accuracy_score, float):
            filename = _pypads_env.call.call_id.context.container.__name__ + "." + _pypads_env.call.call_id.wrappee.__name__ + ".txt"
            try_mlflow_log(mlflow.log_metric, filename, accuracy_score, step=_pypads_env.call.call_id.call_number)
        else:
            logger.warning("Mlflow metrics have to be doubles. Could log the return value of type '" + 
                           str( type(accuracy_score)) + "' of '" + _pypads_env.call.call_id.context.container.__name__ + 
                           "." + _pypads_env.call.call_id.wrappee.__name__ + "' as artifact instead.")
            
        # Let's say we want to log f1_score whenver someone computes the accuracy score just to compare 
        # since accuracy alone can be misleading in some use cases.
        
        from sklearn.metrics import f1_score
        f1 = f1_score(*_args,**_kwargs)
        print("Computing f1_score")
        if isinstance(f1, float):
            filename = _pypads_env.call.call_id.context.container.__name__ + "." + f1_score.__qualname__ + ".CUSTOM_LOGGER.txt"
            try_mlflow_log(mlflow.log_metric, filename, f1, step=_pypads_env.call.call_id.call_number)
        else:
            logger.warning("Mlflow metrics have to be doubles.")

            

##### Custom mapping file to define the hook for our custom logger
* Event : "pypads_metrics"
* Hook : "accuracy_score"

In [2]:
# Custom mapping file 
mapping_json = {
  "default_hooks": {
    "modules": {
      "fns": {}
    },
    "classes": {
      "fns": {
        "pypads_init": [
          "__init__"
        ]
      }
    },
    "fns": {}
  },
  "algorithms": [
    {
      "name": "sklearn classification metrics",
      "other_names": [],
      "implementation": {
        "sklearn": "sklearn.metrics.classification"
      },
      "hooks": {
        "pypads_metric": [
          "accuracy_score"
        ]
      }
    }
  ],
  "metadata": {
    "author": "DEMO",
    "library": "sklearn",
    "library_version": "0.19.1",
    "mapping_version": "0.1"
  }
}

# MappingFile instance
from pypads.autolog.mappings import MappingFile
import json
mapping_example = MappingFile("sklearn_example1", mapping_json)

We should then link the event "pypads_metric" to our new custom logger function by overwriting the currently mapped logging function **Metric** in DEFAULT_LOGGING_FNS. This could be done by passing the following dictionary,

```python
from pypads.base import DEFAULT_LOGGING_FNS
DEFAULT_LOGGING_FNS["metric"] = custom_metric()
```



In [3]:
from pypads.base import DEFAULT_LOGGING_FNS
DEFAULT_LOGGING_FNS["metric"] = custom_metric()
DEFAULT_LOGGING_FNS

{'parameters': <pypads.functions.analysis.validation.parameters.Parameters at 0x7f573b122550>,
 'output': <pypads.functions.loggers.data_flow.Output at 0x7f573b1225f8>,
 'input': <pypads.functions.loggers.data_flow.Input at 0x7f573b1226a0>,
 'hardware': {<pypads.functions.loggers.hardware.Cpu at 0x7f573b122748>,
  <pypads.functions.loggers.hardware.Ram at 0x7f573b1227f0>},
 'metric': <__main__.custom_metric at 0x7f5774622f98>,
 'autolog': <pypads.functions.loggers.mlflow.mlflow_autolog.MlflowAutologger at 0x7f573b1229e8>,
 'pipeline': <pypads.functions.loggers.pipeline_detection.PipelineTracker at 0x7f573b122a90>,
 'log': <pypads.functions.loggers.debug.Log at 0x7f573b122b38>,
 'init': <pypads.functions.loggers.debug.LogInit at 0x7f573b122be0>}

##### Utilities 

In [4]:
# A simple method to see logged data
import http.server
import socketserver
import os
import shutil

def setup_server(path,PORT=8000):
    web_dir = path
    os.chdir(web_dir)

    Handler = http.server.SimpleHTTPRequestHandler
    httpd = socketserver.TCPServer(("", PORT), Handler)
    return httpd

def archive(output_filename,dir_name):
    return shutil.make_archive(output_filename, 'zip', dir_name)

##### Script to test our custom logger

In [5]:
# Temporary directory to store resutls in (Default directory is $HOME/.mlruns/)
from tempfile import TemporaryDirectory
import threading
temp_dir = TemporaryDirectory()

# Initializing PyPads
from padrepads.base import PyPadrePads
custom_logging = {"metric": custom_metric()}
tracker = PyPadrePads(uri=temp_dir.name,mapping=mapping_example, logging_fns=custom_logging)

###### Script #######
from sklearn.metrics import accuracy_score
import numpy as np

# From the previous experiment
test_labels = np.asarray([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1])

preds = np.asarray([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1])

# Computing the accuracy_score tracked by our custom logger
accuracy = accuracy_score(test_labels, preds)

###### getting uri for metrics folder

run = tracker.api.active_run()
metrics = run.info.run_id+'/'+'metrics'

tracker.api.end_run()

##### Logged results (to show the folder structure) #######

path = archive(temp_dir.name+'/logs', temp_dir.name)



server = setup_server(temp_dir.name+'/'+run.info.experiment_id,PORT=8001)

threading.Thread(target=server.serve_forever).start()

from IPython.display import IFrame

frame = IFrame("http://localhost:8001/"+metrics,width=800, height=650)

print('To get the logged results zipped, download them at http://localhost:8001/logs.zip')
###### Logged results #######
tracker.deactivate_tracking()

/home/mehdi/anaconda3/envs/padre37/lib/python3.7/site-packages/networkx/classes/reportviews.py:95: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable


2020-05-26 13:57:29.850 | INFO     | pypads.autolog.mappings:load_mapping:198 - Added mapping file with name: keras_2_3_1.json


2020-05-26 13:57:29.850 | INFO     | pypads.autolog.mappings:load_mapping:198 - Added mapping file with name: keras_2_3_1.json


2020-05-26 13:57:29.852 | INFO     | pypads.autolog.mappings:load_mapping:198 - Added mapping file with name: sklearn_0_19_1.json


2020-05-26 13:57:29.852 | INFO     | pypads.autolog.mappings:load_mapping:198 - Added mapping file with name: sklearn_0_19_1.json


...

2020-05-26 13:57:30.211 | WARNING  | pypads.functions.util:check_index:82 - There are uncommitted changes in your git!


2020-05-26 13:57:30.211 | WARNING  | pypads.functions.util:check_index:82 - There are uncommitted changes in your git!


2020-05-26 13:57:30.236 | WARNING  | pypads.functions.util:check_index:126 - Checking the index of your repository failed due to Cmd('git') failed due to: exit code(1)
  cmdline: git stash push --include-untracked
  stderr: 'usage: git stash list [<options>]
   or: git stash show [<stash>]
   or: git stash drop [-q|--quiet] [<stash>]
   or: git stash ( pop | apply ) [--index] [-q|--quiet] [<stash>]
   or: git stash branch <branchname> [<stash>]
   or: git stash [save [--patch] [-k|--[no-]keep-index] [-q|--quiet]
		       [-u|--include-untracked] [-a|--all] [<message>]]
   or: git stash clear'


2020-05-26 13:57:30.236 | WARNING  | pypads.functions.util:check_index:126 - Checking the index of your repository failed due to Cmd('git') failed due to: exit code(1)
  cmdline: git stash push --include-untracked
  stderr: 'usage: git stash list [<options>]
   or: git stash show [<stash>]
   or: git stash drop [-q|--quiet] [<stash>]
   or: git stash ( pop | apply ) [--index] [-q|--quiet] [<stash>]
   or: git stash branch <branchname> [<stash>]
   or: git stash [save [--patch] [-k|--[no-]keep-index] [-q|--quiet]
		       [-u|--include-untracked] [-a|--all] [<message>]]
   or: git stash clear'


Computing f1_score
To get the logged results zipped, download them at http://localhost:8001/logs.zip


In [6]:
frame

127.0.0.1 - - [26/May/2020 13:57:30] "GET /ee0c316620ca4f46a3c6640b1b7aae32/metrics HTTP/1.1" 301 -
127.0.0.1 - - [26/May/2020 13:57:30] "GET /ee0c316620ca4f46a3c6640b1b7aae32/metrics/ HTTP/1.1" 200 -


As we can see there is a logged f1_score that was computed inside our custom logger.

In [7]:
server.shutdown()
temp_dir.cleanup()